In [14]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math

In [15]:
def process_dataset(dataset_dir):
    data = []
    labels = []
    paths = []

    for sub_folder in os.listdir(dataset_dir):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, sub_folder))
        for i, filename in enumerate(sub_folder_files):
            img_path = os.path.join(dataset_dir, sub_folder, filename)
            img = cv.imread(img_path)
            data.append(img)
            labels.append(sub_folder)
            paths.append(img_path)

    labels = np.array(labels)
    paths = np.array(paths)

    return data, labels, paths

In [16]:
def color_segmentation(images):
    segmented_images = []
    for image in images:
        hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
        lower_green = np.array([36, 40, 40])
        upper_green = np.array([70, 255, 255])
        mask = cv.inRange(hsv, lower_green, upper_green)
        segmented_image = cv.bitwise_and(image, image, mask=mask)
        segmented_images.append(segmented_image)
    return segmented_images

In [17]:
def edge_detection(images):
    edge_images = []
    for image in images:
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        edges = cv.Canny(gray, 50, 255)
        edge_images.append(edges)
    return edge_images

In [18]:
def enhance_image(images):
    enhanced_images = []
    for image in images:
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        equalized_image = cv.equalizeHist(gray)
        enhanced_image = cv.cvtColor(equalized_image, cv.COLOR_GRAY2BGR)
        enhanced_images.append(enhanced_image)
    return enhanced_images

In [19]:
def opening(images, kernel_size):
    opened_images = []
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    for image in images:
        eroded = cv.erode(image, kernel, iterations=1)
        opened = cv.dilate(eroded, kernel, iterations=1)
        opened_images.append(opened)
    return opened_images

In [20]:
def normalize_images(images):
    normalized_images = []
    for img in images:
        normalized_img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX)
        normalized_images.append(normalized_img)
    return normalized_images

In [21]:
dataset = "dataset/"
data, labels, paths = process_dataset(dataset)
segmented_images = color_segmentation(data)

enhanced_images = enhance_image(segmented_images)

normalized_images = normalize_images(enhanced_images)

opened_images = opening(normalized_images, kernel_size=2)

edge_images = edge_detection(opened_images)

In [27]:
def classify_images(edge_images, paths):
    for edge_image, path in zip(edge_images, paths):
        file_name = os.path.basename(path)
        white_pixel_count = np.sum(edge_image > 0)
        threshold_dense = 15000
        threshold_sparse = 5000
        if white_pixel_count > threshold_dense:
            classification = "hutan lebat"
        elif white_pixel_count < threshold_sparse:
            classification = "hutan gundul"
        else:
            classification = "setengah lebat"
        save_dir = os.path.join(dataset, classification)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, file_name)
        cv.imwrite(save_path, edge_image)

classify_images(edge_images, paths)
